In [1]:
from datetime import datetime
import pandas
from typing import List
import pyspark.sql.functions as F

import query_lib

In [2]:
BASE_DIR='./test_files/'
BASE_PATIENT_URL='http://localhost:8099/openmrs/ws/fhir2/R4/Patient/'
patient_query = query_lib.patient_query_factory(
    query_lib.Runner.SPARK, BASE_DIR)

In [3]:
patient_query.include_all_other_codes()
# Note the first call to `find_patient_aggregates` starts a local Spark
# cluster, load input files, and flattens observations. These won't be
# done in subsequent calls of this function on the same instance.
agg_df = patient_query.find_patient_aggregates(BASE_PATIENT_URL)
agg_df

[INDICATORS_LOG 2021-06-24 15:46:32.369345] Number of Patient resources= 1000
[INDICATORS_LOG 2021-06-24 15:46:33.751294] Number of Observation resources= 72869
[INDICATORS_LOG 2021-06-24 15:46:40.250328] Number of flattened obs rows = 72869
[INDICATORS_LOG 2021-06-24 15:46:46.590866] Number of aggregated obs= 19675
[INDICATORS_LOG 2021-06-24 15:46:50.761598] Number of joined patient_agg_obs= 19675


,patientId,birthDate,gender,code,valueCode,num_obs,min_value,max_value,min_date,max_date,min_date_value,max_date_value,min_date_value_code,max_date_value_code
0,00c05602-a2bc-4469-8fe0-5ee94d3e2925,1988-06-05,female,5242AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,None,2,66.0000,87.0000,2019-11-26T08:13:33+00:00,2020-08-08T08:04:33+00:00,2019-11-26T08:13:33+00:00_SeP_66.0000,2020-08-08T08:04:33+00:00_SeP_87.0000,2019-11-26T08:13:33+00:00_SeP_None,2020-08-08T08:04:33+00:00_SeP_None
1,012fc78b-13ad-46b9-bc82-b6021c35159c,1936-03-27,female,1284AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,135462AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,None,None,2020-03-04T09:32:21+00:00,2020-03-04T09:32:21+00:00,2020-03-04T09:32:21+00:00_SeP_None,2020-03-04T09:32:21+00:00_SeP_None,2020-03-04T09:32:21+00:00_SeP_135462AAAAAAAAAA...,2020-03-04T09:32:21+00:00_SeP_135462AAAAAAAAAA...
2,0213330f-8b33-4bad-83c8-b5fd0c180b3b,1980-09-07,female,5089AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,None,3,176.0000,238.0000,2019-03-30T08:37:06+00:00,2020-09-08T07:51:06+00:00,2019-03-30T08:37:06+00:00_SeP_238.0000,2020-09-08T07:51:06+00:00_SeP_186.0000,2019-03-30T08:37:06+00:00_SeP_None,2020-09-08T07:51:06+00:00_SeP_None
3,043807ad-afea-41df-a310-674eeb05a52f,2014-10-27,male,1284AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,119537AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,None,None,2020-03-24T09:03:46+00:00,2020-03-24T09:03:46+00:00,2020-03-24T09:03:46+00:00_SeP_None,2020-03-24T09:03:46+00:00_SeP_None,2020-03-24T09:03:46+00:00_SeP_119537AAAAAAAAAA...,2020-03-24T09:03:46+00:00_SeP_119537AAAAAAAAAA...
4,0654c4e7-22bd-42ca-a2fd-bebcd42bdb8a,2008-08-08,female,159394AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,159393AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,3,None,None,2019-09-09T09:03:28+00:00,2020-04-13T09:56:28+00:00,2019-09-09T09:03:28+00:00_SeP_None,2020-04-13T09:56:28+00:00_SeP_None,2019-09-09T09:03:28+00:00_SeP_159393AAAAAAAAAA...,2020-04-13T09:56:28+00:00_SeP_159393AAAAAAAAAA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19670,eaaa5932-6102-4915-a90f-7ff330c2a0e1,1961-08-27,female,1284AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,119656AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1,None,None,2020-10-02T09:35:06+00:00,2020-10-02T09:35:06+00:00,2020-10-02T09:35:06+00:00_SeP_None,2020-10-02T09:35:06+00:00_SeP_None,2020-10-02T09:35:06+00:00_SeP_119656AAAAAAAAAA...,2020-10-02T09:35:06+00:00_SeP_119656AAAAAAAAAA...
19671,edd5828c-38f0-4020-a64b-c732e05c9144,1971-05-07,female,5242AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,None,8,11.0000,97.0000,2019-01-23T08:23:21+00:00,2020-09-30T08:27:21+00:00,2019-01-23T08:23:21+00:00_SeP_36.0000,2020-09-30T08:27:21+00:00_SeP_64.0000,2019-01-23T08:23:21+00:00_SeP_None,2020-09-30T08:27:21+00:00_SeP_None
19672,ee337278-da4e-4b90-a0a3-0dc95855a885,1959-05-24,male,5088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,None,1,40.0000,40.0000,2020-01-25T08:47:58+00:00,2020-01-25T08:47:58+00:00,2020-01-25T08:47:58+00:00_SeP_40.0000,2020-01-25T08:47:58+00:00_SeP_40.0000,2020-01-25T08:47:58+00:00_SeP_None,2020-01-25T08:47:58+00:00_SeP_None
19673,f16b5191-af47-4c5a-b9ca-71e0a4365824,1998-10-02,female,159946AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,159944AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,3,None,None,2019-07-30T09:50:25+00:00,2020-05-04T09:28:26+00:00,2019-07-30T09:50:25+00:00_SeP_None,2020-05-04T09:28:26+00:00_SeP_None,2019-07-30T09:50:25+00:00_SeP_159944AAAAAAAAAA...,2020-05-04T09:28:26+00:00_SeP_159944AAAAAAAAAA...


In [4]:
obs = patient_query._obs_df
#obs.withColumn('coding', F.explode('code.coding')).head()
#obs.withColumn('coding', F.explode('code.coding')).filter(F.col('coding.system').isNull()).head()
obs.withColumn('coding', F.explode('code.coding')).filter('coding.system IS NULL').head()

Row(id='http://localhost:8099/openmrs/ws/fhir2/R4/Observation/14c0fede-464f-425a-9289-8e127ec15b23', meta=None, implicitRules=None, language=None, text=None, contained=None, identifier=None, basedOn=None, status='final', category=[Row(id=None, coding=[Row(id=None, system='http://terminology.hl7.org/CodeSystem/observation-category', version=None, code='exam', display='Exam', userSelected=None)], text=None)], code=Row(id=None, coding=[Row(id=None, system=None, version=None, code='5242AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Respiratory rate', userSelected=None)], text=None), subject=Row(DeviceId=None, GroupId=None, LocationId=None, PatientId='0f5c5b9a-c02f-4c07-a7b7-66705d7716bf', id=None, reference='Patient/0f5c5b9a-c02f-4c07-a7b7-66705d7716bf', identifier=None, display='Elizabeth Mitchell (OpenMRS ID: 1008YN)'), context=Row(EncounterId='984c7a46-1ca6-4617-a3fc-6f60bc11015b', EpisodeOfCareId=None, id=None, reference='Encounter/984c7a46-1ca6-4617-a3fc-6f60bc11015b', identifier=None, d

In [5]:
patient_query._flat_obs.head()

Row(coding=Row(id=None, system=None, version=None, code='5242AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Respiratory rate', userSelected=None), valueCoding=None, value=Row(quantity=Row(id=None, value=Decimal('93.0000'), comparator=None, unit=None, system=None, code=None), codeableConcept=None, string=None, boolean=None, range=None, ratio=None, sampledData=None, attachment=None, time=None, dateTime=None, period=None), patientId='0f5c5b9a-c02f-4c07-a7b7-66705d7716bf', dateTime='2020-11-04T08:41:04+00:00', dateAndValue='2020-11-04T08:41:04+00:00_SeP_93.0000', dateAndValueCode='2020-11-04T08:41:04+00:00_SeP_None')

In [6]:
patient_query._flat_obs.filter('coding.code = "5090AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"').head()

Row(coding=Row(id=None, system=None, version=None, code='5090AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', display='Height (cm)', userSelected=None), valueCoding=None, value=Row(quantity=Row(id=None, value=Decimal('168.0000'), comparator=None, unit=None, system=None, code=None), codeableConcept=None, string=None, boolean=None, range=None, ratio=None, sampledData=None, attachment=None, time=None, dateTime=None, period=None), patientId='c309ea1d-64be-4079-b19b-fe441efe18f2', dateTime='2020-06-19T08:09:05+00:00', dateAndValue='2020-06-19T08:09:05+00:00_SeP_168.0000', dateAndValueCode='2020-06-19T08:09:05+00:00_SeP_None')

In [8]:
def _find_age_band(birth_date: str, end_date: datetime) -> str:
  """Given the birth date, finds the age_band for PEPFAR disaggregation."""
  age = None
  try:
    # TODO handle all different formats (issues #174)
    birth = datetime.strptime(birth_date, '%Y-%m-%d')
    age = int((end_date - birth).days / 365.25)
  except Exception as e:
    common.custom_log('Invalid birth_date format: {}'.format(e))
    age = 999999

  if age == 999999:
    return 'ERROR'
  if age < 1:
    return '0-1'
  if age <= 4:
    return '1-4'
  if age <= 9:
    return '5-9'
  if age <= 14:
    return '10-14'
  if age <= 19:
    return '15-19'
  if age <= 24:
    return '20-24'
  if age <= 49:
    return '25-49'
  return '50+'


def _agg_buckets(birth_date: str, gender: str, end_date: datetime) -> List[str]:
  """Generates the list of all PEPFAR disaggregation buckets."""
  age_band = _find_age_band(birth_date, end_date)
  return [age_band + '_' + gender, 'ALL-AGES_' + gender,
          age_band + '_ALL-GENDERS', 'ALL-AGES_ALL-GENDERS']

def calc_TX_PVLS(patient_agg_obs: pandas.DataFrame, VL_code: str,
    failure_threshold: int, end_date_str: str = None) -> pandas.DataFrame:
  """Calculates TX_PVLS indicator with its corresponding disaggregations.

  Args:
    patient_agg_obs: An output from `patient_query.find_patient_aggregates()`.
    VL_code: The code for viral load values.
    failure_threshold: VL count threshold of failure.
    end_date: The string representation of the last date as 'YYYY-MM-DD'.
  Returns:
    The aggregated DataFrame.
  """
  end_date = datetime.today()
  if end_date_str:
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
  temp_df = patient_agg_obs[(patient_agg_obs['code'] == VL_code)].copy()
  # Note the above copy is used to avoid setting a new column on a slice next:
  temp_df['sup_VL'] = (temp_df['max_value'] < failure_threshold)
  temp_df['buckets'] = temp_df.apply(
      lambda x: _agg_buckets(x.birthDate, x.gender, datetime.today()), axis=1)
  ex_temp_df = temp_df.explode('buckets')
  #VL_agg_P['ratio'] = VL_agg_P['count']/num_patients
  return ex_temp_df.groupby(['sup_VL', 'buckets'], as_index=False).size()
    
VL_CODE = '5090AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
calc_TX_PVLS(agg_df, VL_CODE, 150, end_date_str='2020-12-30')

,sup_VL,buckets,size
0,False,1-4_ALL-GENDERS,18
1,False,1-4_female,11
2,False,1-4_male,7
3,False,10-14_ALL-GENDERS,32
4,False,10-14_female,15
5,False,10-14_male,17
6,False,15-19_ALL-GENDERS,34
7,False,15-19_female,19
8,False,15-19_male,15
9,False,20-24_ALL-GENDERS,46


In [9]:
patient_agg_obs = agg_df
VL_code = '5090AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
failure_threshold = 150
temp_df = patient_agg_obs[(patient_agg_obs['code'] == VL_code) &
                          (patient_agg_obs['max_value'] < failure_threshold)].copy()
# Note the above copy is used to avoid setting a new column on a slice next:
temp_df['buckets'] = temp_df.apply(
    lambda x: _agg_buckets(x.birthDate, x.gender, datetime.today()), axis=1)
ex_temp_df = temp_df.explode('buckets')
print('After explode: {}'.format(ex_temp_df.index.size))
final_df = ex_temp_df.groupby('buckets', as_index=False).size()
final_df

After explode: 580


,buckets,size
0,1-4_ALL-GENDERS,3
1,1-4_female,1
2,1-4_male,2
3,10-14_ALL-GENDERS,5
4,10-14_female,3
5,10-14_male,2
6,15-19_ALL-GENDERS,8
7,15-19_female,3
8,15-19_male,5
9,20-24_ALL-GENDERS,6
